In [9]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from BaselineRemoval import BaselineRemoval
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
from torch.utils.data import TensorDataset
import torch
from torch import nn
import sys
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from torch.utils.data import TensorDataset


In [10]:
DATALABEL = ['ear lobe','inner arm','thumb nail','median cubital vein']
WATCH_PROB_PREDICT = False ## if you want to observe the probability of each choice
PLT = '/root/projects/ProjectAug22/CP/Diabete_detection_withRamanSpec/dataset/' ## change to the path to dataset
PLT_mac = '/Volumes/ExternalSSDForMac/AITLecture/CSforDSAIAUG22/_CP_project/DataScienceAndMLProjects/Diabete_detection_withRamanSpec/dataset'

SMOOTH = False

SEED = 25811243



In [11]:
from src import spectrumloader as spl
from src import get_torch_loader as tlg
# from src.modeltorch_template import NeuralNet,RamConv1d,RamLSTM,RamConv1d_bnmx,IntANN,IntRamLSTM,RamConv1d_mx
from src import trainResult_show as mtp

In [19]:
print(len(y_test0))

4


In [18]:
standard,data_all = spl.read_file('dataset/')
Xs,ys,_ = spl.cut_tonumpy(data_all)
assert len(Xs) == 4, 'DATA error'

_,X_testall,_,y_testall = spl.split_train_test(Xs,ys)
X_test_VR = spl.fluoresence_removal(X_testall)
X_test0_std,X_test1_std,X_test2_std,X_test3_std,normalizer2 = spl.seting_normalized_fuoresence_smoothing(True,True,X_test_VR)

assert len(normalizer2) == 4, 'Normalizer is not here'

y_test0,y_test1,y_test2,y_test3 = y_testall

ds_test_earloop = TensorDataset(torch.FloatTensor(X_test0_std),torch.LongTensor(y_test0))
ds_test_inarm = TensorDataset(torch.FloatTensor(X_test1_std),torch.LongTensor(y_test1))
ds_test_thumbnail = TensorDataset(torch.FloatTensor(X_test2_std),torch.LongTensor(y_test2))
ds_test_vain = TensorDataset(torch.FloatTensor(X_test3_std),torch.LongTensor(y_test3))
batch_size =4 

test_loader0 = torch.utils.data.DataLoader(dataset=ds_test_earloop,batch_size=batch_size,shuffle=True)
test_loader1 = torch.utils.data.DataLoader(dataset=ds_test_inarm,batch_size=batch_size,shuffle=True)
test_loader2 = torch.utils.data.DataLoader(dataset=ds_test_thumbnail,batch_size=batch_size,shuffle=True)
test_loader3 = torch.utils.data.DataLoader(dataset=ds_test_vain,batch_size=batch_size,shuffle=True)


test_loader_all = [test_loader0,test_loader1,test_loader2,test_loader3]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    for each_loc_test, each_loc_model in zip(test_loader_all,"LISTOFMODEL"):
        correct = 0
        for i, (spec, y) in enumerate(each_loc_test):  

            spec = spec.to(device)
            y_test = y.to(device)

            y_hat = each_loc_model(spec)
            predicted = torch.max(y_hat,1)[1]
            correct += (predicted == y_test).sum()
        print(f'Test accuracy: {correct.item()}/{len(y_test0)} = {correct.item()*100/(len(y_test0)):7.3f}%')

# if SMOOTH:
#     X_test_VR = spl.savitzky_smooth(X_test_VR)

In [13]:
# X_trainall,X_testall,y_trainall,y_testall = spl.split_train_test(Xs,ys)
# y_train0,y_train1,y_train2,y_train3 = y_trainall

In [14]:
# X_train_VR = spl.fluoresence_removal(X_trainall)
# if SMOOTH:
#     X_train_VR = spl.savitzky_smooth(X_train_VR)

In [15]:
X_test_VR = spl.fluoresence_removal(X_testall)
if SMOOTH:
    X_test_VR = spl.savitzky_smooth(X_test_VR)

In [16]:
# X_train0_std,X_train1_std,X_train2_std,X_train3_std,normalizer = spl.seting_normalized_fuoresence_smoothing(True,True,X_train_VR)


# X_test0_std,X_test1_std,X_test2_std,X_test3_std,normalizer2 = spl.seting_normalized_fuoresence_smoothing(True,True,X_test_VR)


# assert len(normalizer) == 4, 'Normalizer is not here'

In [17]:
y_test0,y_test1,y_test2,y_test3 = y_testall